In [5]:
import sys; sys.path.append("../")

import pandas as pd

from lib.explore import show_unique
from lib.match import ThresholdMatcher, JaroWinklerSimilarity, ColumnsIndex, StringSimilarity
from clean.baton_rouge_so_cprr import clean18

In [2]:
df = clean18()
df

../clean/baton_rouge_so_cprr.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  names1 = df.name.str.strip().str.replace(
../lib/clean.py:152: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../clean/baton_rouge_so_cprr.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  rules = df.infraction.str.replace(
../clean/baton_rouge_so_cprr.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "rule_code"] = rules.iloc[:, 1]\
../lib/clean.py:128: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.strip().str.lower()\
../lib/clean.py:120: FutureWarning: The default value of regex will change from True to False in a future version.
  df

,badge_no,rank_desc,race,sex,birth_year,complainant_type,disposition,action,department_desc,first_name,...,rank_year,rank_month,rank_day,occur_year,occur_month,occur_day,occur_time,data_production_year,agency,uid
0,2206,sergeant,white,male,1984,public,founded,forwarded completed report to administration f...,uniform patrol,william,...,2017,2,6,2018,1,26,21:00,2018,Baton Rouge SO,402619644d9eff018cd39a757bcc5153
1,1711,corporal,white,male,1984,administration,founded,7 day suspension; 6 month loss of take-home ve...,scat,eric,...,2012,2,13,2018,2,20,00:00,2018,Baton Rouge SO,7b22c2ab899789ff98f5cd884b4175be
2,1308,lieutenant,white,male,1968,coworker,founded,resigned,parish prison (training),eddie,...,2006,9,6,2018,3,12,00:00,2018,Baton Rouge SO,1f86bca13a125c7422d192a14159ac21
3,1534,sergeant,black,male,1966,coworker,inconclusive,none,construction services,troy,...,2015,6,15,2018,3,15,00:00,2018,Baton Rouge SO,93c7b5cba8c235cb5aa61b013922d71a
4,2527,deputy,black,female,1986,public,founded,verbal warning from captain flynn about courte...,community policing,pashion,...,2014,12,16,2018,4,8,15:30,2018,Baton Rouge SO,6cf20697312e7736147155175876da95
5,1797,corporal,white,female,1982,supervisor,founded,none,financial crimes,leighann,...,2015,2,9,2018,4,11,10:00,2018,Baton Rouge SO,b91b655b93a3c0de5c46674b3472867f
6,769,captain,black,female,1965,subordinate,unfounded,none,communications,eleanor,...,2014,6,16,2018,4,13,12:00,2018,Baton Rouge SO,d4abd7442439b47d115132746ecdf7da
7,2067,deputy,white,male,1982,public,inconclusive,none,scat,john,...,2008,11,17,2018,4,19,00:00,2018,Baton Rouge SO,b180787c744ce3e7383aa683f40b51ea
8,1519,lieutenant,white,male,1982,administration,founded,5 day suspension,uniform patrol,justin,...,2017,11,27,2018,4,22,00:00,2018,Baton Rouge SO,3f6d99799d87868c8b9f69126f4b331b
9,2363,sergeant,white,male,1983,public,unfounded,none,uniform patrol,dustin,...,2017,11,27,2018,5,10,00:00,2018,Baton Rouge SO,93efca0952d59008ceb2f8f0baaeb7ac


In [4]:
df.columns

Index(['badge_no', 'rank_desc', 'race', 'sex', 'birth_year',
       'complainant_type', 'disposition', 'action', 'department_desc',
       'first_name', 'middle_initial', 'last_name', 'rule_violation',
       'rule_code', 'rank_year', 'rank_month', 'rank_day', 'occur_year',
       'occur_month', 'occur_day', 'occur_time', 'data_production_year',
       'agency', 'uid'],
      dtype='object')

In [3]:
pprr = pd.read_csv("../data/match/pprr_baton_rouge_csd_2019.csv")
pprr

,year,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,annual_salary,employee_id,data_production_year,hire_year,hire_month,hire_day,term_year,term_month,term_day,agency,uid
0,2018,williams,decory,l,5040,police-operational serv bureau,2070,senior stock clerk,active,26541.06,0xF1ABD670358E036C31296E66B3B66C382AC00812,2019,2014,6,23,NaN,NaN,NaN,Baton Rouge CSD,2c26c1da182e017154760576bb26d178
1,2018,weissberg,deena,j,5020,police-uniform patrol bureau,5005,police officer,active,53024.53,0xAF3E133428B9E25C55BC59FE534248E6A0C0F17B,2019,2010,3,31,NaN,NaN,NaN,Baton Rouge CSD,b4911ac24a815588b44f634a2994652b
2,2018,graham,michael,h,5050,police-communications,5115,police comm officer i-42 hrs,inactive,54243.63,0x9109C85A45B703F87F1413A405549A2CEA9AB556,2019,2007,4,21,2019.0,1.0,22.0,Baton Rouge CSD,375b7460fced4f1c25800ab70aad3539
3,2018,gillich,joshua,m,5070,police-criminal investigation,5005,police officer,active,28738.00,0xB4C96D80854DD27E76D8CC9E21960EEBDA52E962,2019,2008,12,8,NaN,NaN,NaN,Baton Rouge CSD,f3a681bf5b2ea66f984df7d95ae390e1
4,2018,saleh,haled,NaN,5020,police-uniform patrol bureau,5005,police officer,inactive,62144.25,0x1F1362EA41D1BC65BE321C0A378A20159F9A26D0,2019,2006,6,12,2019.0,4.0,26.0,Baton Rouge CSD,bd0a7112d1ce9caf09c00dd75e759411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,2018,howard,walter,c,5060,police-special operations,6525,school crossing guard,active,942.50,0x0B9918551C179FAF20295B3C0CB7798486824D12,2019,2018,10,24,NaN,NaN,NaN,Baton Rouge CSD,b809606018d8c135d42348fe0ddc7457
1958,2018,houston,stacy,m,5060,police-special operations,6525,school crossing guard,active,928.00,0x08AB94787936E21703BA8ABB75A6B57237E83FF4,2019,2018,10,24,NaN,NaN,NaN,Baton Rouge CSD,cad7ff3de443661681277b85450a209d
1959,2018,dyson,kenneth,w,5060,police-special operations,6525,school crossing guard,active,928.00,0x6BE2349E0ECE2FA121CA321B3C85CCA02311E62C,2019,2018,10,24,NaN,NaN,NaN,Baton Rouge CSD,4720d7b49d294945e5fd8dcb9ba129bd
1960,2018,augustus,robert,l,5060,police-special operations,6525,school crossing guard,inactive,913.50,0x9E065978592E8F4AE463B70C70C8A3C1ED6F9389,2019,2018,10,24,2019.0,6.0,10.0,Baton Rouge CSD,bf23de2bb605f07db3277192bce6ca28


In [16]:
dfa = df[['first_name', 'middle_initial', 'last_name', 'uid']]\
    .drop_duplicates().set_index('uid')
dfa.loc[:, 'fnc'] = dfa.first_name.map(lambda x: x[:1])
dfb = pprr[['first_name', 'middle_initial', 'last_name', 'uid']]\
    .drop_duplicates().set_index('uid')
dfb.loc[:, 'fnc'] = dfb.first_name.map(lambda x: x[:1])
matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(['fnc']), {
    'middle_initial': StringSimilarity(),
    'first_name': JaroWinklerSimilarity(),
    'last_name': JaroWinklerSimilarity(),
})

In [17]:
matcher.get_sample_pairs()

first_name  \
score_range pair_idx sim_score row_key                                       
0.90-0.85   0        0.855868  3ab64d2e2b554256b897419613449ce9    kenneth   
                               c4bc4091dd6fc360f072f0a342fbb34d    kenneth   
            1        0.854506  3ab64d2e2b554256b897419613449ce9    kenneth   
                               88d46892244448c0c461772bd8b93f0d    kenneth   
0.85-0.80   0        0.840829  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               6ec24ae49802e9aed5d80c4f01aae815      duane   
0.80-0.75   0        0.779257  3ab64d2e2b554256b897419613449ce9    kenneth   
                               d0104f31c1ce247fe4657c45c950d28c      kevin   
            1        0.777897  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               ea13bafbc9b534603c40c9161c90588a      devin   
            2        0.776899  3ab64d2e2b554256b897419613449ce9    kenneth   
                               ea2b5e20e35ec27c9e05162b6fe01d45      kynon   
            3        0.753519  5d99e8da3c3ab0a8c67482d5b5aa708e   courtney   
                               8500a21fcf09593156206b7fe325b702    charles   
0.75-0.70   0        0.718280  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               3c2c90890cfb85e018c7f7654bbe3b3c     darren   
            1        0.718280  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               82af7aafb18adb62c8484e4b8ee023fa     demika   
            2        0.718036  3ab64d2e2b554256b897419613449ce9    kenneth   
                               02a64c51c655f6864b4e8050ff2d8b66       kama   
            3        0.708083  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               63d9340124154db6b4056976e20d6adb    deborah   
            4        0.704746  3ab64d2e2b554256b897419613449ce9    kenneth   
                               54a6391284cb6e84c714d47fcacc4ab1    kenneth   

                                                                middle_initial  \
score_range pair_idx sim_score row_key                                           
0.90-0.85   0        0.855868  3ab64d2e2b554256b897419613449ce9              j   
                               c4bc4091dd6fc360f072f0a342fbb34d              j   
            1        0.854506  3ab64d2e2b554256b897419613449ce9              j   
                               88d46892244448c0c461772bd8b93f0d              j   
0.85-0.80   0        0.840829  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               6ec24ae49802e9aed5d80c4f01aae815              l   
0.80-0.75   0        0.779257  3ab64d2e2b554256b897419613449ce9              j   
                               d0104f31c1ce247fe4657c45c950d28c              j   
            1        0.777897  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               ea13bafbc9b534603c40c9161c90588a              l   
            2        0.776899  3ab64d2e2b554256b897419613449ce9              j   
                               ea2b5e20e35ec27c9e05162b6fe01d45              j   
            3        0.753519  5d99e8da3c3ab0a8c67482d5b5aa708e              r   
                               8500a21fcf09593156206b7fe325b702              r   
0.75-0.70   0        0.718280  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               3c2c90890cfb85e018c7f7654bbe3b3c              l   
            1        0.718280  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               82af7aafb18adb62c8484e4b8ee023fa              l   
            2        0.718036  3ab64d2e2b554256b897419613449ce9              j   
                               02a64c51c655f6864b4e8050ff2d8b66              j   
            3        0.708083  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               63d9340124154db6b4056976e20d6adb              l   
            4        0.704746  3ab64d2e2b554256b897419613449ce9    